In [8]:
import os
import numpy as np
import math

data_root = os.getcwd()
label_path = label_path = os.path.join(data_root, 'training', 'lidar', f'{75469:06d}.yaml')

In [9]:
import yaml
with open(label_path, 'r') as f:
        data=yaml.safe_load(f)
annotation = {}
annotation['P0']=data['lidar0_pose']
annotation['P1']=data['lidar1_pose']
annotation['P2']=data['lidar2_pose']
annotation['P3']=data['lidar3_pose']
#data['lidar0_pose']
lidar_pose_ref = annotation['P0']
lidar_pose_ref

[-47.5255126953125,
 -10.45819091796875,
 5.555450439453125,
 0.0,
 -90.48019409179688,
 0.0]

In [10]:
v = data['vehicles']
v = v[146]
v

{'angle': [0.0, -90.34283447265625, 0.0],
 'center': [0.004043041728436947, -2.1788478079542983e-06, 0.718859851360321],
 'extent': [2.4508416652679443, 1.0641621351242065, 0.7553732395172119],
 'location': [-44.17448043823242, -35.31916046142578, 0.05999999865889549]}

In [22]:
# testing bounding box
def create_bbx(extent):
    """
    Create bounding box with 8 corners under obstacle vehicle reference.
    Parameters
    ----------
    extent : list
        Width, height, length of the bbx.
    Returns
    -------
    bbx : np.array
        The bounding box with 8 corners, shape: (8, 3)
    """

    bbx = np.array([[extent[0], -extent[1], -extent[2]],
                    [extent[0], extent[1], -extent[2]],
                    [-extent[0], extent[1], -extent[2]],
                    [-extent[0], -extent[1], -extent[2]],
                    [extent[0], -extent[1], extent[2]],
                    [extent[0], extent[1], extent[2]],
                    [-extent[0], extent[1], extent[2]],
                    [-extent[0], -extent[1], extent[2]]])

    return bbx
extent = v['extent']
# shape (3, 8)
bbx = create_bbx(extent).T
# bounding box under ego coordinate shape (4, 8)
bbx = np.r_[bbx, [np.ones(bbx.shape[1])]]
bbx.shape

(4, 8)

In [17]:
def x_to_world(pose):
    """
    The transformation matrix from x-coordinate system to carla world system
    Parameters
    ----------
    pose : list
        [x, y, z, roll, yaw, pitch]
    Returns
    -------
    matrix : np.ndarray
        The transformation matrix.
    """
    x, y, z, roll, yaw, pitch = pose[:]

    # used for rotation matrix
    c_y = np.cos(np.radians(yaw))
    s_y = np.sin(np.radians(yaw))
    c_r = np.cos(np.radians(roll))
    s_r = np.sin(np.radians(roll))
    c_p = np.cos(np.radians(pitch))
    s_p = np.sin(np.radians(pitch))

    matrix = np.identity(4)
    # translation matrix
    matrix[0, 3] = x
    matrix[1, 3] = y
    matrix[2, 3] = z

    # rotation matrix
    matrix[0, 0] = c_p * c_y
    matrix[0, 1] = c_y * s_p * s_r - s_y * c_r
    matrix[0, 2] = -c_y * s_p * c_r - s_y * s_r
    matrix[1, 0] = s_y * c_p
    matrix[1, 1] = s_y * s_p * s_r + c_y * c_r
    matrix[1, 2] = -s_y * s_p * c_r + c_y * s_r
    matrix[2, 0] = s_p
    matrix[2, 1] = -c_p * s_r
    matrix[2, 2] = c_p * c_r

    return matrix

lidar_to_world_transform = x_to_world(lidar_pose_ref)
world_to_lidar_transform = np.linalg.inv(lidar_to_world_transform)
world_to_lidar_transform.shape

(4, 4)

In [24]:
"""
# https://github.com/DerrickXuNu/OpenCOOD/blob/84703477cdf57f61056f1a646e4b0a167f7a77d7/opencood/utils/box_utils.py#L435
bbx_lidar = np.dot(world_to_lidar_transform, bbx).T
bbx_lidar.shape
bbx_lidar = np.expand_dims(bbx_lidar[:, :3], 0)
bbx_lidar.shape
""""

(1, 8, 3)

In [43]:
location_in_carlaworld = v['location']
location_in_carlaworld

[-44.17448043823242, -35.31916046142578, 0.05999999865889549]

In [35]:
location_in_carlaworld = v['location']
location_in_carlaworld = location_in_carlaworld +[1]
location_in_carlaworld = np.array(location_in_carlaworld)
location_in_carlaworld = np.expand_dims(location_in_carlaworld, axis=0).T
location_in_carlaworld.shape
location_in_carlaworld

array([[-44.17448044],
       [-35.31916046],
       [  0.06      ],
       [  1.        ]])

In [37]:
location_in_lidar = np.dot(world_to_lidar_transform,location_in_carlaworld)
print(location_in_carlaworld)
print(lidar_pose_ref)
print(location_in_lidar)

[[-44.17448044]
 [-35.31916046]
 [  0.06      ]
 [  1.        ]]
[-47.5255126953125, -10.45819091796875, 5.555450439453125, 0.0, -90.48019409179688, 0.0]
[[24.83201186]
 [ 3.55927112]
 [-5.49545044]
 [ 1.        ]]


In [66]:
location_in_lidar.shape
location_in_lidar_squeeze = location_in_lidar.squeeze()
location_in_lidar_squeeze.shape
location_in_lidar_x = location_in_lidar_squeeze[0]
location_in_lidar_x

#location_in_lidar_y = list(location_in_lidar[1].squeeze())
#location_in_lidar_z = list(location_in_lidar[2].squeeze())
#location_in_lidar_list = location_in_lidar_x+location_in_lidar_y+location_in_lidar_z

24.832011858398058

In [ ]:
# convention in roll/yaw/pitch (rotation in with x axis/z axis/y axisLhttps://carla.readthedocs.io/en/latest/python_api/#carla.Rotation)
rotation_in_carlaworld = v['']

In [ ]:


def x_to_world(pose):
    """
    The transformation matrix from x-coordinate system to carla world system
    Parameters
    ----------
    pose : list
        [x, y, z, roll, yaw, pitch]
    Returns
    -------
    matrix : np.ndarray
        The transformation matrix.
    """
    x, y, z, roll, yaw, pitch = pose[:]

    # used for rotation matrix
    c_y = np.cos(np.radians(yaw))
    s_y = np.sin(np.radians(yaw))
    c_r = np.cos(np.radians(roll))
    s_r = np.sin(np.radians(roll))
    c_p = np.cos(np.radians(pitch))
    s_p = np.sin(np.radians(pitch))

    matrix = np.identity(4)
    # translation matrix
    matrix[0, 3] = x
    matrix[1, 3] = y
    matrix[2, 3] = z

    # rotation matrix
    matrix[0, 0] = c_p * c_y
    matrix[0, 1] = c_y * s_p * s_r - s_y * c_r
    matrix[0, 2] = -c_y * s_p * c_r - s_y * s_r
    matrix[1, 0] = s_y * c_p
    matrix[1, 1] = s_y * s_p * s_r + c_y * c_r
    matrix[1, 2] = -s_y * s_p * c_r + c_y * s_r
    matrix[2, 0] = s_p
    matrix[2, 1] = -c_p * s_r
    matrix[2, 2] = c_p * c_r

    return matrix

def location_carlaworld2lidar(location_in_carlaworld,lidar_pose):
    """
    project locations in carld world coordinates to lidar coordinate
    Note lidar can be substitute by the pose of any objects under carla world 

    """
    lidar_to_world_transform = x_to_world(lidar_pose)
    world_to_lidar_transform = np.linalg.inv(lidar_to_world_transform)

    location_in_lidar = 

    return location_in_lidar

NameError: name 'v' is not defined